In [4]:
# Colab-safe NCERT ZIP parser: put everything in ONE cell

import re
import io
import json
import zipfile
from typing import Optional, Dict, Any, List

try:
    import PyPDF2  # pip install PyPDF2
except ImportError:
    PyPDF2 = None

CLASS_MAP = {chr(ord('a') + i): f"Class {i+1}" for i in range(12)}  # a..l => Class 1..12
MEDIUM_MAP = {'e': 'English', 'h': 'Hindi', 'u': 'Urdu'}

SUBJECT_MAP = {
    'mh': 'Mathematics',
    'ph': 'Physics',
    'ch': 'Chemistry',
    'bo': 'Biology',
    'cs': 'Computer Science',
    'ip': 'Informatics Practices',
    'sc': 'Science',
    'lm': 'Lab Manual',
    # Extend as you encounter more NCERT codes (e.g., ec, ge, hi, ps, etc.)
}

FILENAME_RE = re.compile(
    r'^(?P<class>[a-l])(?P<medium>[ehu])(?P<subject>[a-z]{2})(?P<part>[12])(?P<chap>(\d{2}|[a-z]{2}))$',
    re.IGNORECASE
)

def parse_code(basename: str) -> Optional[Dict[str, Any]]:
    m = FILENAME_RE.match(basename)
    if not m:
        return None
    gd = m.groupdict()
    class_letter = gd['class'].lower()
    medium_letter = gd['medium'].lower()
    subject_code = gd['subject'].lower()
    part = int(gd['part'])
    chap_raw = gd['chap'].lower()

    info = {
        'class': CLASS_MAP.get(class_letter, f'Unknown ({class_letter})'),
        'medium': MEDIUM_MAP.get(medium_letter, f'Unknown ({medium_letter})'),
        'subject_code': subject_code,
        'subject': SUBJECT_MAP.get(subject_code, f'Unknown ({subject_code})'),
        'part': part,
        'chapter': None,
        'section_type': 'chapter'
    }

    if chap_raw.isdigit():
        info['chapter'] = int(chap_raw)
    else:
        if chap_raw == 'an':
            info['section_type'] = 'answers'
        elif chap_raw == 'ps':
            info['section_type'] = 'prelims'
        else:
            info['section_type'] = chap_raw

    return info

def extract_titles_from_pdf_bytes(pdf_bytes: bytes) -> Dict[str, Optional[str]]:
    if PyPDF2 is None:
        return {'book_title': None, 'chapter_title': None}
    try:
        with io.BytesIO(pdf_bytes) as bio:
            reader = PyPDF2.PdfReader(bio)
            if not reader.pages:
                return {'book_title': None, 'chapter_title': None}
            text = reader.pages[0].extract_text() or ""
            lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
            chap_line = next((ln for ln in lines if ln.lower().startswith('chapter') or re.match(r'^\d+\s', ln)), None)
            book_line = None
            for ln in lines[:20]:
                if len(ln) >= 6 and (ln.isupper() or any(k in ln.lower() for k in [
                    'mathematics','physics','chemistry','biology','computer science','informatics','science'
                ])):
                    book_line = ln
                    break
            return {'book_title': book_line, 'chapter_title': chap_line}
    except Exception:
        return {'book_title': None, 'chapter_title': None}

def describe_entry(name: str, pdf_bytes: Optional[bytes]) -> Dict[str, Any]:
    base = name.rsplit('/', 1)[-1]
    stem = base.lower().rsplit('.', 1)[0]
    parsed = parse_code(stem)
    result = {'file': base, 'parsed': parsed, 'book_title': None, 'chapter_title': None}

    if pdf_bytes:
        titles = extract_titles_from_pdf_bytes(pdf_bytes)
        result.update(titles)

    if parsed:
        subj = parsed['subject']
        label_parts = [parsed['class'], parsed['medium'], subj]
        if parsed['section_type'] == 'chapter' and parsed['chapter'] is not None:
            label_parts.append(f"Part {parsed['part']}, Chapter {parsed['chapter']:02d}")
        else:
            label_parts.append(f"Part {parsed['part']}, {parsed['section_type'].title()}")
        result['label_from_code'] = ' | '.join(label_parts)
    else:
        result['label_from_code'] = 'Unrecognized filename pattern'

    if result.get('book_title') or result.get('chapter_title'):
        title_label = []
        if result.get('book_title'):
            title_label.append(result['book_title'])
        if result.get('chapter_title'):
            title_label.append(result['chapter_title'])
        result['label_from_pdf'] = ' | '.join(title_label)

    return result

def parse_zip(zip_path: str) -> List[Dict[str, Any]]:
    out = []
    with zipfile.ZipFile(zip_path, 'r') as zf:
        for zi in zf.infolist():
            if zi.is_dir():
                continue
            if not zi.filename.lower().endswith('.pdf'):
                continue
            with zf.open(zi, 'r') as f:
                pdf_bytes = f.read()
            out.append(describe_entry(zi.filename, pdf_bytes))
    return out

def run_zip(zip_path: str, json_out='ncert_index.json', csv_out='ncert_index.csv'):
    rows = parse_zip(zip_path)
    import csv
    with open(json_out, 'w', encoding='utf-8') as f:
        json.dump(rows, f, ensure_ascii=False, indent=2)

    fieldnames = ['file', 'label_from_code', 'label_from_pdf',
                  'book_title', 'chapter_title',
                  'class', 'medium', 'subject_code', 'subject', 'part', 'chapter', 'section_type']
    with open(csv_out, 'w', encoding='utf-8', newline='') as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            pinfo = r.get('parsed') or {}
            w.writerow({
                'file': r.get('file'),
                'label_from_code': r.get('label_from_code'),
                'label_from_pdf': r.get('label_from_pdf'),
                'book_title': r.get('book_title'),
                'chapter_title': r.get('chapter_title'),
                'class': pinfo.get('class'),
                'medium': pinfo.get('medium'),
                'subject_code': pinfo.get('subject_code'),
                'subject': pinfo.get('subject'),
                'part': pinfo.get('part'),
                'chapter': pinfo.get('chapter'),
                'section_type': pinfo.get('section_type'),
            })
    print(f"Wrote {len(rows)} entries to {json_out} and {csv_out}")

# Notebook-friendly CLI block: ignores unknown flags like "-f <file>"
def main_notebook_safe():
    import argparse
    p = argparse.ArgumentParser(description='Parse NCERT ZIP of chapter PDFs into structured names')
    p.add_argument('zipfile', nargs='?', help='Path to NCERT ZIP containing chapter PDFs')
    p.add_argument('--json', default='ncert_index.json', help='Write JSON index here')
    p.add_argument('--csv', default='ncert_index.csv', help='Write CSV index here')
    args, _ = p.parse_known_args()  # ignore hidden notebook flags
    if args.zipfile:
        run_zip(args.zipfile, json_out=args.json, csv_out=args.csv)
    else:
        p.print_help()

# Uncomment to use CLI-like behavior inside a notebook cell:
# main_notebook_safe()

# Recommended in notebooks: call run_zip directly, e.g.:
# run_zip('/content/NCERT_Textbooks.zip', json_out='index.json', csv_out='index.csv')


In [10]:
# Interactive decoder for NCERT filename codes (Jupyter/Colab-friendly)

import re

CLASS_MAP = {chr(ord('a') + i): f"Class {i+1}" for i in range(12)}  # a..l => Class 1..12
MEDIUM_MAP = {'e': 'English', 'h': 'Hindi', 'u': 'Urdu'}
# Update your SUBJECT_MAP in both the interactive decoder and the ZIP parser
MEDIUM_MAP = {
    'e': 'English',
    'h': 'Hindi',
    'u': 'Urdu',
    's': 'Sanskrit',
}

SUBJECT_MAP = {
    # Mathematics
    'mh': 'Mathematics',          # kemh106, jemh101
    'jm': 'Mathematics',          # aejm1ps (Joyful Mathematics)

    # Science umbrellas and specific sciences
    'sc': 'Science',              # jesc..., iesc...
    'ph': 'Physics',              # leph...
    'ch': 'Chemistry',            # lech...
    'bo': 'Biology',              # lebo...
    'bt': 'Biotechnology',        # kebt...
    'lm': 'Lab Manual',           # kelm...

    # Social sciences (VI–X umbrella and senior-secondary splits)
    'ss': 'Social Science',       # iess..., hess...
    'es': 'Social Science',       # Class 6 Exploring Society (fees1xx)
    'gy': 'Geography',            # legy..., kegy...
    'hs': 'History',              # kehs...
    'ps': 'Political Science',    # leps...
    'ec': 'Economics',            # keec...

    # Commerce
    'ac': 'Accountancy',          # keac...
    'bs': 'Business Studies',     # lebs...

    # Computing
    'cs': 'Computer Science',     # lecs...
    'ip': 'Informatics Practices',# keip...

    # Humanities electives
    'py': 'Psychology',           # lepy..., kepy...
    'sy': 'Sociology',            # lesy..., kesy...
    'he': 'Home Science',         # lehe...

    # English readers (named series)
    'sp': 'English (Supplementary Reader)',  # kesp... Snapshots

    # Languages
    'de': 'Sanskrit',             # fsde...
}
# NEW (accept any medium letter; labeling still controlled by MEDIUM_MAP):
FILENAME_RE = re.compile(r'^(?P<class>[a-l])(?P<medium>[a-z])(?P<subject>[a-z]{2})(?P<part>[12])(?P<chap>(\d{2}|[a-z]{2}))$', re.IGNORECASE)



def decode_code(code: str):
    stem = code.strip().lower()
    if stem.endswith('.pdf'):
        stem = stem[:-4]
    m = FILENAME_RE.match(stem)
    if not m:
        return None
    gd = m.groupdict()
    class_letter = gd['class']
    medium_letter = gd['medium']
    subject_code = gd['subject']
    part = int(gd['part'])
    chap_raw = gd['chap']

    info = {
        'class': CLASS_MAP.get(class_letter, f'Unknown ({class_letter})'),
        'medium': MEDIUM_MAP.get(medium_letter, f'Unknown ({medium_letter})'),
        'subject_code': subject_code,
        'subject': SUBJECT_MAP.get(subject_code, f'Unknown ({subject_code})'),
        'part': part,
        'chapter': None,
        'section_type': 'chapter'
    }
    if chap_raw.isdigit():
        info['chapter'] = int(chap_raw)
    else:
        if chap_raw == 'an':
            info['section_type'] = 'answers'
        elif chap_raw == 'ps':
            info['section_type'] = 'prelims'
        else:
            info['section_type'] = chap_raw
    return info

def explain(code: str):
    info = decode_code(code)
    if not info:
        print("Unrecognized code pattern; expected like lech101 or jemh1an")
        return
    if info['section_type'] == 'chapter' and info['chapter'] is not None:
        print(f"{info['class']} | {info['medium']} | {info['subject']} | Part {info['part']}, Chapter {info['chapter']:02d}")
    else:
        print(f"{info['class']} | {info['medium']} | {info['subject']} | Part {info['part']}, {info['section_type'].title()}")

# Interactive loop
while True:
    s = input("Enter NCERT code (e.g., lech101 or lech101.pdf). Press Enter to quit: ").strip()
    if not s:
        break
    explain(s)


Enter NCERT code (e.g., lech101 or lech101.pdf). Press Enter to quit: fsde101
Class 6 | Sanskrit | Sanskrit | Part 1, Chapter 01
Enter NCERT code (e.g., lech101 or lech101.pdf). Press Enter to quit: 
